In [1]:
import warnings

import pandas as pd


def load_data(
    file_path, input_text_column, target_text_column, label_column, keep_label=1
):
    df = pd.read_csv(file_path, sep="\t", error_bad_lines=False)
    df = df.loc[df[label_column] == keep_label]
    df = df.rename(
        columns={input_text_column: "input_text", target_text_column: "target_text"}
    )
    df = df[["input_text", "target_text"]]
    df["prefix"] = "paraphrase"

    return df


def clean_unnecessary_spaces(out_string):
    if not isinstance(out_string, str):
        warnings.warn(f">>> {out_string} <<< is not a string.")
        out_string = str(out_string)
    out_string = (
        out_string.replace(" .", ".")
        .replace(" ?", "?")
        .replace(" !", "!")
        .replace(" ,", ",")
        .replace(" ' ", "'")
        .replace(" n't", "n't")
        .replace(" 'm", "'m")
        .replace(" 's", "'s")
        .replace(" 've", "'ve")
        .replace(" 're", "'re")
    )
    return out_string

In [2]:
import os
from datetime import datetime
import logging

import pandas as pd
from sklearn.model_selection import train_test_split
from simpletransformers.seq2seq import Seq2SeqModel, Seq2SeqArgs

#from utils import load_data, clean_unnecessary_spaces


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.ERROR)

2023-04-09 15:55:33.635119: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-09 15:55:33.767784: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-09 15:55:33.770459: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-09 15:55:33.770472: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

In [3]:
train_df = pd.read_csv("easy_train.csv",encoding="latin-1").astype(str)
eval_df = pd.read_csv("easy_valid.csv",encoding="latin-1").astype(str)
print(eval_df)
train_df.columns=['ID','target_text','input_text','tokens']
eval_df.columns=['ID','target_text','input_text','tokens']

     Unnamed: 0                                                 tp  \
0      26305507   List all the vaccine recipients who took test...   
1      33109235   What is the number of records that the vaccin...   
2      27306117   Return all the cases where the vaccine recipi...   
3       7101494   Provide me all the cases completed the form o...   
4      23304823  Give me all the patients who took test of bloo...   
...         ...                                                ...   
1299   32408835   How many records had POLIO VIRUS, INACT. and ...   
1300    3200896   Give me all the patients who is 106.0 years o...   
1301   10102357  Give me all the patients who died on 12/18/2021.    
1302   43311709   What is the number of the cases where the vac...   
1303   44411886  Give me all the patients who got HPV (GARDASIL...   

                                                     nl  \
0     List of all vaccines taking Testostrone gel du...   
1     How many vaccines have been recorde

In [5]:
print([column for column in eval_df])
eval_df.iat[2,0]

['ID', 'target_text', 'input_text', 'tokens']


'27306117'

In [6]:
# train_df = train_df.loc[train_df["label"] == "1"]
# eval_df = eval_df.loc[eval_df["label"] == "1"]
train_df = train_df[["target_text","input_text"]]
eval_df = eval_df[["target_text","input_text"]]

train_df["prefix"] = "paraphrase"
eval_df["prefix"] = "paraphrase"

# # MSRP Data
# train_df = pd.concat(
#     [
#         train_df,
#         load_data("data/msr_paraphrase_train.txt", "#1 String", "#2 String", "Quality"),
#     ]
# )
# eval_df = pd.concat(
#     [
#         eval_df,
#         load_data("data/msr_paraphrase_test.txt", "#1 String", "#2 String", "Quality"),
#     ]
# )

# # Quora Data

# # The Quora Dataset is not separated into train/test, so we do it manually the first time.
# df = load_data(
#     "data/quora_duplicate_questions.tsv", "question1", "question2", "is_duplicate"
# )
# q_train, q_test = train_test_split(df)

# q_train.to_csv("data/quora_train.tsv", sep="\t")
# q_test.to_csv("data/quora_test.tsv", sep="\t")

# # The code block above only needs to be run once.
# # After that, the two lines below are sufficient to load the Quora dataset.

# # q_train = pd.read_csv("data/quora_train.tsv", sep="\t")
# # q_test = pd.read_csv("data/quora_test.tsv", sep="\t")

# train_df = pd.concat([train_df, q_train])
# eval_df = pd.concat([eval_df, q_test])

# train_df = train_df[["prefix", "input_text", "target_text"]]
# eval_df = eval_df[["prefix", "input_text", "target_text"]]

# train_df = train_df.dropna()
# eval_df = eval_df.dropna()

# train_df["input_text"] = train_df["input_text"].apply(clean_unnecessary_spaces)
# train_df["target_text"] = train_df["target_text"].apply(clean_unnecessary_spaces)

# eval_df["input_text"] = eval_df["input_text"].apply(clean_unnecessary_spaces)
# eval_df["target_text"] = eval_df["target_text"].apply(clean_unnecessary_spaces)

print(train_df)

                                             target_text  \
0       How many patients has a blood group after vac...   
1       On 03/25/2021, how many people have been reco...   
2       How many vaccine recipients got MNQ? which ME...   
3       Find the number of cases that the vaccine rec...   
4      Give me all the patients who got vaccine from ...   
...                                                  ...   
10427   List all the patients who got VACCINE NOT SPE...   
10428   What is the number of cases where the interva...   
10429   Find all the records where the adverse event ...   
10430      How many patients passed away on 11/15/2021.    
10431   Find any cases where the vaccine recipients w...   

                                              input_text      prefix  
0      A number of patients had a blood type after th...  paraphrase  
1          On 03/25/2021, how many people were recorded?  paraphrase  
2      How many vaccines have MNQ? Which is most popu...  paraphra

In [7]:
model_args = Seq2SeqArgs()
model_args.do_sample = True
model_args.eval_batch_size = 64
model_args.evaluate_during_training = True
model_args.evaluate_during_training_steps = 2500
model_args.evaluate_during_training_verbose = True
model_args.fp16 = False
model_args.learning_rate = 5e-5
model_args.max_length = 128
model_args.max_seq_length = 128
#model_args.num_beams = None
model_args.num_return_sequences = 3
model_args.num_train_epochs = 2
model_args.overwrite_output_dir = True
model_args.reprocess_input_data = True
model_args.save_eval_checkpoints = False
model_args.save_steps = -1
model_args.top_k = 50
model_args.top_p = 0.95
model_args.train_batch_size = 8
model_args.use_multiprocessing = False
model_args.wandb_project = "Paraphrasing with BART"


model = Seq2SeqModel(
    encoder_decoder_type="bart",
    encoder_decoder_name="facebook/bart-large",
    args=model_args,
    use_cuda=False
)

model.train_model(train_df, eval_data=eval_df)

INFO:simpletransformers.seq2seq.seq2seq_utils: Creating features from dataset file at cache_dir/


  0%|          | 0/10432 [00:00<?, ?it/s]

INFO:simpletransformers.seq2seq.seq2seq_model: Training started


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


...

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

  0%|          | 0/1304 [00:00<?, ?it/s]

	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


INFO:simpletransformers.seq2seq.seq2seq_model:{'eval_loss': 0.05154648902160781}
INFO:simpletransformers.seq2seq.seq2seq_model:Saving model into outputs/
INFO:simpletransformers.seq2seq.seq2seq_model: Training of facebook/bart-large model complete. Saved to outputs/.


(2608,
 {'global_step': [1304, 2500, 2608],
  'eval_loss': [0.07581928691693715, 0.05126631490531422, 0.05154648902160781],
  'train_loss': [0.03716167062520981,
   0.01603108085691929,
   0.026486210525035858]})

In [8]:
results=[]
ground_truth=[]


In [9]:
to_predict = [
    prefix + ": " + str(input_text)
    for prefix, input_text in zip(eval_df["prefix"].tolist(), eval_df["input_text"].tolist())
]
truth = eval_df["target_text"].tolist()

preds = model.predict(to_predict)

# Saving the predictions if needed
os.makedirs("predictions", exist_ok=True)

with open(f"predictions/predictions_{datetime.now()}.txt", "w") as f:
    for i, text in enumerate(eval_df["input_text"].tolist()):
        f.write(str(text) + "\n\n")

        f.write("Truth:\n")
        f.write(truth[i] + "\n\n")
        ground_truth.append(truth[i])
        f.write("Prediction:\n")
        for pred in preds[i]:
            f.write(str(pred) + "\n")
            results.append(pred)
        f.write(
            "________________________________________________________________________________\n"
        )

Exception in thread SystemMonitor:
Traceback (most recent call last):
  File "/home/wenlong/anaconda3/lib/python3.9/threading.py", line 980, in _bootstrap_inner
    self.run()
  File "/home/wenlong/anaconda3/lib/python3.9/threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "/home/wenlong/anaconda3/lib/python3.9/site-packages/wandb/sdk/internal/system/system_monitor.py", line 118, in _start
    asset.start()
  File "/home/wenlong/anaconda3/lib/python3.9/site-packages/wandb/sdk/internal/system/assets/cpu.py", line 166, in start
    self.metrics_monitor.start()
  File "/home/wenlong/anaconda3/lib/python3.9/site-packages/wandb/sdk/internal/system/assets/interfaces.py", line 168, in start
    logger.info(f"Started {self._process.name}")
AttributeError: 'NoneType' object has no attribute 'name'


Generating outputs:   0%|          | 0/21 [00:00<?, ?it/s]

In [10]:
results_1=[]

for i in range(0,len(results),3):
    results_1.append(results[i])

    
print(len(results_1))    
    

1304


In [46]:
# test2_df = pd.read_csv("valid.csv",encoding="latin-1").astype(str)
# test2_df.columns=['input_text','target_text','ground_truth']


# test2_df = test2_df[["target_text",'ground_truth']]
# test2_df["prefix"] = "paraphrase"
# test2_df

,target_text,ground_truth,prefix
0,List all the vaccine recipients who took testo...,List all the vaccine recipients who took testo...,paraphrase
1,What is the number of records that the vaccine...,What is the number of records that the vaccine...,paraphrase
2,Return all the cases where the vaccine recipie...,Return all the cases where the vaccine recipie...,paraphrase
3,Provide me all the cases completed the form on...,Provide me all the cases completed the form on...,paraphrase
4,Give me all the patients who took test of arte...,Give me all the patients who took test of bloo...,paraphrase
...,...,...,...
1299,"How many records had POLIO VIRUS, INACT. and D...","How many records had POLIO VIRUS, INACT. and D...",paraphrase
1300,Give me all the patients who is 106.0 years old.,Give me all the patients who is 106.0 years old.,paraphrase
1301,Give me all the patients who died on 12/18/2021.,Give me all the patients who died on 12/18/2021.,paraphrase
1302,What is the number of the cases where the vacc...,What is the number of the cases where the vacc...,paraphrase


In [11]:
model_args.do_sample = True
model_args.num_beams = None
model_args.num_return_sequences = 3
model_args.max_length = 128
model_args.top_k = 50
model_args.top_p = 0.95

In [ ]:
import logging

from simpletransformers.seq2seq import Seq2SeqModel


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.ERROR)

model = Seq2SeqModel(
    encoder_decoder_type="bart", encoder_decoder_name="outputs",use_cuda=False
)


while True:
    original = input("Enter text to paraphrase: ")
    to_predict = [original]

    preds = model.predict(to_predict)

    print("---------------------------------------------------------")
    print(original)

    print()
    print("Predictions >>>")
    for pred in preds[0]:
        print(pred)

    print("---------------------------------------------------------")
    print()

Enter text to paraphrase: How many of those vaccined may suffer from traumatic brain injury when they are vaccinated 


Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

---------------------------------------------------------
How many of those vaccined may suffer from traumatic brain injury when they are vaccinated 

Predictions >>>
 How many vaccine recipients are dignosed of traumatic brain injury at the time of vaccination? 
 What is the number of vaccine recipients that had traumatic brain injury when they took the shot? 
 What is the number of vaccine recipients that had traumatic brain injury when they took the shot? 
---------------------------------------------------------



In [28]:
test2_df.iat[0,0]

'How many patients are dignosed of anxiety?'

In [47]:
# import re
# from rouge_score import rouge_scorer
# scorer = rouge_scorer.RougeScorer(['rouge1'])
# means_score=[]
# precision=[]
# precision.clear()
# for i in range(len(test2_df)):
#     a = scorer.score(test2_df.iat[i,0],test2_df.iat[i,1])
#     print(a)
#     b = re.findall('(?<=precision=)\d+\.\d+?',str(a))
#     precision.append(b)
# #    means_score.append(scorer.score(test2_df[i,0],test2_df[i,1]))
# #   means_score_f=means_score_f+means_score
# #means_score_f=means_score_f/len(ground_truth)
# precision

INFO:absl:Using default tokenizer.


{'rouge1': Score(precision=1.0, recall=1.0, fmeasure=1.0)}
{'rouge1': Score(precision=1.0, recall=1.0, fmeasure=1.0)}
{'rouge1': Score(precision=1.0, recall=1.0, fmeasure=1.0)}
{'rouge1': Score(precision=1.0, recall=1.0, fmeasure=1.0)}
{'rouge1': Score(precision=0.75, recall=0.8181818181818182, fmeasure=0.7826086956521738)}
{'rouge1': Score(precision=1.0, recall=1.0, fmeasure=1.0)}
{'rouge1': Score(precision=1.0, recall=1.0, fmeasure=1.0)}
{'rouge1': Score(precision=1.0, recall=1.0, fmeasure=1.0)}
{'rouge1': Score(precision=1.0, recall=1.0, fmeasure=1.0)}
{'rouge1': Score(precision=0.8571428571428571, recall=0.8571428571428571, fmeasure=0.8571428571428571)}
{'rouge1': Score(precision=1.0, recall=1.0, fmeasure=1.0)}
{'rouge1': Score(precision=0.8, recall=0.8888888888888888, fmeasure=0.8421052631578948)}
{'rouge1': Score(precision=0.7272727272727273, recall=0.6666666666666666, fmeasure=0.6956521739130435)}
{'rouge1': Score(precision=1.0, recall=1.0, fmeasure=1.0)}
{'rouge1': Score(precis

{'rouge1': Score(precision=0.9230769230769231, recall=0.9230769230769231, fmeasure=0.9230769230769231)}
{'rouge1': Score(precision=1.0, recall=1.0, fmeasure=1.0)}
{'rouge1': Score(precision=0.8181818181818182, recall=0.9, fmeasure=0.8571428571428572)}
{'rouge1': Score(precision=1.0, recall=1.0, fmeasure=1.0)}
{'rouge1': Score(precision=0.75, recall=0.8571428571428571, fmeasure=0.7999999999999999)}
{'rouge1': Score(precision=1.0, recall=1.0, fmeasure=1.0)}
{'rouge1': Score(precision=1.0, recall=1.0, fmeasure=1.0)}
{'rouge1': Score(precision=1.0, recall=1.0, fmeasure=1.0)}
{'rouge1': Score(precision=1.0, recall=1.0, fmeasure=1.0)}
{'rouge1': Score(precision=0.8333333333333334, recall=1.0, fmeasure=0.9090909090909091)}
{'rouge1': Score(precision=1.0, recall=1.0, fmeasure=1.0)}
{'rouge1': Score(precision=1.0, recall=1.0, fmeasure=1.0)}
{'rouge1': Score(precision=0.8888888888888888, recall=1.0, fmeasure=0.9411764705882353)}
{'rouge1': Score(precision=0.9166666666666666, recall=0.91666666666

[['1.0'],
 ['1.0'],
 ['1.0'],
 ['1.0'],
 ['0.7'],
 ['1.0'],
 ['1.0'],
 ['1.0'],
 ['1.0'],
 ['0.8'],
 ['1.0'],
 ['0.8'],
 ['0.7'],
 ['1.0'],
 ['1.0'],
 ['1.0'],
 ['0.8'],
 ['0.8'],
 ['0.9'],
 ['0.8'],
 ['0.8'],
 ['0.9'],
 ['0.7'],
 ['0.8'],
 ['0.9'],
 ['1.0'],
 ['0.8'],
 ['0.9'],
 ['0.9'],
 ['1.0'],
 ['1.0'],
 ['0.9'],
 ['1.0'],
 ['1.0'],
 ['1.0'],
 ['1.0'],
 ['1.0'],
 ['1.0'],
 ['0.7'],
 ['1.0'],
 ['0.8'],
 ['0.7'],
 ['0.8'],
 ['1.0'],
 ['0.8'],
 ['0.6'],
 ['0.8'],
 ['0.8'],
 ['1.0'],
 ['0.9'],
 ['1.0'],
 ['0.8'],
 ['1.0'],
 ['1.0'],
 ['0.8'],
 ['1.0'],
 ['0.6'],
 ['1.0'],
 ['1.0'],
 ['0.5'],
 ['0.8'],
 ['1.0'],
 ['1.0'],
 ['0.8'],
 ['1.0'],
 ['0.8'],
 ['1.0'],
 ['0.7'],
 ['0.9'],
 ['1.0'],
 ['1.0'],
 ['0.8'],
 ['1.0'],
 ['1.0'],
 ['1.0'],
 ['1.0'],
 ['1.0'],
 ['1.0'],
 ['0.8'],
 ['0.6'],
 ['1.0'],
 ['0.7'],
 ['0.8'],
 ['1.0'],
 ['0.8'],
 ['1.0'],
 ['0.8'],
 ['0.9'],
 ['1.0'],
 ['0.8'],
 ['0.9'],
 ['1.0'],
 ['0.7'],
 ['1.0'],
 ['0.8'],
 ['1.0'],
 ['1.0'],
 ['0.8'],
 ['0.9'],
 ['1.0'],


In [40]:
precision[1][0]

'1.0'

In [78]:
import re
from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rouge1'])
means_score=[]
precision=[]
for i in range(len(ground_truth)):
    a=scorer.score(results_1[i],ground_truth[i])
    b = re.findall('(?<=precision=)((-?\d+)(\.\d+)?)',str(a))
    precision.append(b)
    means_score.append(scorer.score(results_1[i],ground_truth[i]))
#   means_score_f=means_score_f+means_score
#means_score_f=means_score_f/len(ground_truth)
precision


INFO:absl:Using default tokenizer.


[[('1.0', '1', '.0')],
 [('1.0', '1', '.0')],
 [('0.9090909090909091', '0', '.9090909090909091')],
 [('1.0', '1', '.0')],
 [('0.9166666666666666', '0', '.9166666666666666')],
 [('0.875', '0', '.875')],
 [('1.0', '1', '.0')],
 [('1.0', '1', '.0')],
 [('0.75', '0', '.75')],
 [('0.8571428571428571', '0', '.8571428571428571')],
 [('1.0', '1', '.0')],
 [('0.0', '0', '.0')],
 [('0.7272727272727273', '0', '.7272727272727273')],
 [('1.0', '1', '.0')],
 [('1.0', '1', '.0')],
 [('1.0', '1', '.0')],
 [('0.8181818181818182', '0', '.8181818181818182')],
 [('0.8571428571428571', '0', '.8571428571428571')],
 [('0.75', '0', '.75')],
 [('1.0', '1', '.0')],
 [('0.875', '0', '.875')],
 [('0.9166666666666666', '0', '.9166666666666666')],
 [('0.7777777777777778', '0', '.7777777777777778')],
 [('0.8888888888888888', '0', '.8888888888888888')],
 [('0.3076923076923077', '0', '.3076923076923077')],
 [('1.0', '1', '.0')],
 [('0.75', '0', '.75')],
 [('0.9090909090909091', '0', '.9090909090909091')],
 [('0.857142

In [241]:
 means_score

[{'rouge1': Score(precision=0.875, recall=1.0, fmeasure=0.9333333333333333)},
 {'rouge1': Score(precision=1.0, recall=1.0, fmeasure=1.0)},
 {'rouge1': Score(precision=1.0, recall=1.0, fmeasure=1.0)},
 {'rouge1': Score(precision=1.0, recall=1.0, fmeasure=1.0)},
 {'rouge1': Score(precision=0.875, recall=0.875, fmeasure=0.875)},
 {'rouge1': Score(precision=1.0, recall=1.0, fmeasure=1.0)},
 {'rouge1': Score(precision=1.0, recall=1.0, fmeasure=1.0)},
 {'rouge1': Score(precision=1.0, recall=1.0, fmeasure=1.0)},
 {'rouge1': Score(precision=0.9375, recall=0.7894736842105263, fmeasure=0.8571428571428572)},
 {'rouge1': Score(precision=0.9090909090909091, recall=1.0, fmeasure=0.9523809523809523)},
 {'rouge1': Score(precision=1.0, recall=1.0, fmeasure=1.0)},
 {'rouge1': Score(precision=0.08333333333333333, recall=0.3333333333333333, fmeasure=0.13333333333333333)},
 {'rouge1': Score(precision=1.0, recall=1.0, fmeasure=1.0)},
 {'rouge1': Score(precision=0.8461538461538461, recall=0.9166666666666666,

In [79]:
eval_df_1 = pd.read_csv("easy_valid.csv",encoding="latin-1").astype(str)
eval_df_1.iat[1,0]

'33109235'

In [80]:
import pandas as pd

df1 = pd.DataFrame(columns = ['ID','results','ground_truth','rouge1'])

for i in range(len(ground_truth)):
    
    df1.loc[i] = {'ID':eval_df_1.iat[i,0],'results':results_1[i],'ground_truth':ground_truth[i],'rouge1':precision[i][0][0]}

df1

,ID,results,ground_truth,rouge1
0,26305507,List all the vaccine recipients who took test...,List all the vaccine recipients who took test...,1.0
1,33109235,What is the number of records that the vaccin...,What is the number of records that the vaccin...,1.0
2,27306117,Return all the cases where the vaccine recipi...,Return all the cases where the vaccine recipi...,0.9090909090909091
3,7101494,Provide me all the cases completed the form o...,Provide me all the cases completed the form o...,1.0
4,23304823,Give me all the patients who took test of elev...,Give me all the patients who took test of bloo...,0.9166666666666666
...,...,...,...,...
1299,32408835,"How many records had VIRUS, INACT. and DT ADS...","How many records had POLIO VIRUS, INACT. and ...",0.9090909090909091
1300,3200896,Give me all the patients who is 106.0 years old.,Give me all the patients who is 106.0 years o...,1.0
1301,10102357,Give me all the patients who died on 12/18/20...,Give me all the patients who died on 12/18/2021.,1.0
1302,43311709,What is the number of the cases where the vac...,What is the number of the cases where the vac...,1.0


In [73]:
# import pandas as pd

# df1 = pd.DataFrame(columns = ['results','ground_truth','rouge1'])

# for i in range(len(ground_truth)):
    
#     df1.loc[i] = {'results':test2_df.iat[i,0],'ground_truth':test2_df.iat[i,1],'rouge1':precision[i][0]}

# df1

In [81]:
fial_ground=[]
for i in range(len(ground_truth)):
    if float(df1.iat[i,3])>0.9:
        fial_ground.append(df1.iloc[i])
fial_ground


[ID                                                       26305507
 results          List all the vaccine recipients who took test...
 ground_truth     List all the vaccine recipients who took test...
 rouge1                                                        1.0
 Name: 0, dtype: object,
 ID                                                       33109235
 results          What is the number of records that the vaccin...
 ground_truth     What is the number of records that the vaccin...
 rouge1                                                        1.0
 Name: 1, dtype: object,
 ID                                                       27306117
 results          Return all the cases where the vaccine recipi...
 ground_truth     Return all the cases where the vaccine recipi...
 rouge1                                         0.9090909090909091
 Name: 2, dtype: object,
 ID                                                        7101494
 results          Provide me all the cases completed t

In [82]:
import pandas as pd
from pandas.core.frame import DataFrame
df_f=DataFrame(fial_ground)
df_f.to_excel('bart_valid.xlsx')

In [245]:
from torchmetrics.functional.text.rouge import rouge_score
score=rouge_score(results_1,ground_truth,accumulate='avg')
score

{'rouge1_fmeasure': tensor(0.8489),
 'rouge1_precision': tensor(0.8590),
 'rouge1_recall': tensor(0.8462),
 'rouge2_fmeasure': tensor(0.7865),
 'rouge2_precision': tensor(0.7949),
 'rouge2_recall': tensor(0.7827),
 'rougeL_fmeasure': tensor(0.8430),
 'rougeL_precision': tensor(0.8531),
 'rougeL_recall': tensor(0.8401),
 'rougeLsum_fmeasure': tensor(0.8438),
 'rougeLsum_precision': tensor(0.8539),
 'rougeLsum_recall': tensor(0.8409)}